# Test Correspondences

In [ ]:
%matplotlib widget
import numpy as np
import os
from plotly.subplots import make_subplots

import planeslam.io as io
from planeslam.general import NED_to_ENU
from planeslam.scan import pc_to_scan
from planeslam.registration import extract_corresponding_features, get_correspondences

os.environ['KMP_DUPLICATE_LIB_OK']='True'

%load_ext autoreload
%autoreload 2

Read in airsim LiDAR and pose data

In [ ]:
# Read in point cloud data
binpath = os.path.join(os.getcwd(), '..', '..', 'data', 'airsim', 'blocks_20_samples_1', 'lidar', 'Drone0')
PC_data = io.read_lidar_bin(binpath)

In [ ]:
# Read in ground-truth poses (in drone local frame)
posepath = os.path.join(os.getcwd(), '..', '..', 'data', 'airsim', 'blocks_20_samples_1', 'poses', 'Drone0')
drone_positions, drone_orientations = io.read_poses(posepath)

In [ ]:
# Convert to ENU
num_scans = len(PC_data)

for i in range(num_scans):
    PC_data[i] = NED_to_ENU(PC_data[i])

Extract planes 

In [ ]:
scans = num_scans * [None]

for i in range(num_scans):
    scans[i] = pc_to_scan(PC_data[i])

In [ ]:
# Plot 2 scans
source = scans[11]
target = scans[12]

fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'surface'}, {'type': 'surface'}]])

for t in source.plot_trace(show_normals=True):
    fig.add_trace(t, row=1, col=1)

for t in target.plot_trace(show_normals=True):
    fig.add_trace(t, row=1, col=2)

fig.update_layout(width=1500, height=600, scene=dict(aspectmode='data'), scene2=dict(aspectmode='data'))
fig.show()

correspondences = get_correspondences(source, target)
print(correspondences)

In [ ]:
from planeslam.registration import so3_residual, so3_jacobian, so3_expmap

n_s, d_s, n_t, d_t = extract_corresponding_features(source, target, correspondences)

# Rotation estimation
R_hat = np.eye(3)

n_iters = 5
lmbda = 1e-8
mu = 1.0

for i in range(n_iters):
    r, n_q = so3_residual(R_hat, n_s, n_t)
    #print("loss: ", np.linalg.norm(r)**2)
    J = so3_jacobian(n_q)
    dw = - mu * np.linalg.inv(J.T @ J + lmbda*np.eye(3)) @ J.T @ r
    R_hat = so3_expmap(dw.flatten()) @ R_hat

r, _ = so3_residual(R_hat, n_s, n_t)
print("final rotation loss: ", np.linalg.norm(r)**2)

# Translation estimation
Rn_s = (R_hat @ n_s.reshape((3, -1), order='F'))
t_hat = np.linalg.lstsq(Rn_s.T, d_s - d_t, rcond=None)[0]

print(R_hat)
print(t_hat)